In [1]:
!pwd

/Users/vascomeerman/Coding/Harvard/CS215/Project/Style-Transfer-App


In [1]:
import time
import sys
import pprint
from argparse import Namespace
import numpy as np
from PIL import Image
import torch
import torchvision.transforms as transforms

# sys.path.append(".")
# sys.path.append("..")


# from datasets import augmentations
from custom_psp.utils.common import tensor2im, log_input_image
from custom_psp.models.psp import pSp

ImportError: dlopen(/Users/vascomeerman/miniforge3/envs/MLOpsProject/lib/python3.9/site-packages/cv2.cpython-39-darwin.so, 10): Library not loaded: @rpath/libvpx.7.dylib
  Referenced from: /Users/vascomeerman/miniforge3/envs/MLOpsProject/lib/libavcodec.58.134.100.dylib
  Reason: no suitable image found.  Did find:
	/Users/vascomeerman/miniforge3/envs/MLOpsProject/lib/libvpx.7.dylib: mach-o, but not built for platform macOS
	/Users/vascomeerman/miniforge3/envs/MLOpsProject/lib/libvpx.7.dylib: mach-o, but not built for platform macOS
	/Users/vascomeerman/miniforge3/envs/MLOpsProject/lib/python3.9/site-packages/../../libvpx.7.dylib: mach-o, but not built for platform macOS
	/Users/vascomeerman/miniforge3/envs/MLOpsProject/lib/libvpx.7.dylib: mach-o, but not built for platform macOS
	/Users/vascomeerman/miniforge3/envs/MLOpsProject/bin/../lib/libvpx.7.dylib: mach-o, but not built for platform macOS

In [ ]:
# Path to the psp repo in our shared folder (I believe)
psp_path = "/content/drive/MyDrive/StyleTransferProject/PSP/v3_pixel2style2pixel"

In [ ]:
# Output of shape 1000, confidence scores for each of the imagenet classes
# Now we will save this model.
import torch.onnx

# # I am not using a GPU here, if you are, move it to cuda
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# device = "cpu"

def export_onnx_model(model, onnx_model_path):
    with torch.no_grad():
        # Bring full model to cpu for conversion
        # model.to(device)

        # Format input image into a batch format the model can use
        # inp_batch = transformed_image1.unsqueeze(0)
        inp_batch = torch.randn(1, 3, 256, 256).to(device)


        # Run the model once on the same input we give it for onnx to run
        # _, _ = model(img_tensor.float(), randomize_noise=False, return_latents= True)
        print("\n\n --------------ONNXX LETS GO ------------------\n\n")

        # Inputs needed for onnx to run 1 inference session so it can determine the graph it needs to convert
        inputs = (inp_batch.float(),
                  {"randomize_noise": False,
                  "return_latents": True})
        
        # Convert our pytroch model to onnx, so we can later on convert it to tf
        torch.onnx.export(model,                                            
                          inputs,
                          onnx_model_path,                                  
                          opset_version=14,
                          do_constant_folding=True,
                          # See: https://github.com/pytorch/fairseq/issues/3395
                          # operator_export_type=torch.onnx.OperatorExportTypes.ONNX_ATEN_FALLBACK,
                          export_params=True,
                          input_names=['input'],
                          output_names=['output'],               
                          # input_names=['input_ids',                         
                          #              'input_code',
                          #              'return_latents']
                          )
        
        print("ONNX Model exported to {0}".format(onnx_model_path))



def export_new_model():
    ckpt2 = torch.load(model_path, map_location='cpu')

    # update the training options
    opts2 = ckpt2['opts']
    if device == "cpu":
        opts2['device'] = "cpu"

    opts2['checkpoint_path'] = model_path
    if 'learn_in_w' not in opts2:
        opts2['learn_in_w'] = False
    if 'output_size' not in opts2:
        opts2['output_size'] = 1024


    opts2 = Namespace(**opts2)
    new_net = pSp(opts2)
    new_net.eval()

    if device != "cpu":
        new_net.cuda() # Don't move model to cuda for conversion
    else:
    print(f"No cuda -> Running on {device}")

    export_onnx_model(new_net, "../output/psp_clean_cpu3.onnx")